# Asimov: Simple Example

This notebook details a simple setup for using the Asimov framework in your own analysis.
A dataset from the Higgs Boson Machine Learning Challenge is used for the demo*.


*Data accessed from Higgs Boson Machine Learning Challenge
http://opendata.cern.ch/record/328
on 18 September 2018 (converted from .csv to .root for this example)

In [1]:
import os
import sys
from time import strftime,localtime

### Import Necessary Modules
Update python paths to include the directories for Asimov and hepPlotter -- subject to how you have checked out these repositories

In [2]:
cwd = os.getcwd()
hpd = cwd.rstrip("examples")+"python/"
if hpd not in sys.path:
    sys.path.insert(0,hpd)
hpd2 = cwd.replace("asimov/examples","hepPlotter/python/")
if hpd2 not in sys.path:
    sys.path.insert(0,hpd2)

In [3]:
import util
from training import Training
from config import Config
import example_plotlabels as plb

/Users/demarley/anaconda2/lib/python2.7/site-packages/matplotlib/style/core.py:51: UserWarning: Style includes a parameter, 'backend', that is not related to style.  Ignoring
  "to style.  Ignoring".format(key))
Using TensorFlow backend.


The python file `example_plotlabels.py` contains an example for how you can organize information concerning sample and variable labels on your plots.  
The objects in this file (`Sample` & `Variable`) are assumed to be available in the Asimov framework.

### Setup Configuration

In [4]:
config   = Config("example_config.txt")  # Set options for asimov
vb       = util.VERBOSE()                # Tool that handles print statements to the console
vb.level = config.verbose_level
vb.initialize()

# Create a new (unique) directory to store the outputs
# I use the current date/time, but this is just a personal preference
date = strftime("%d%b%Y-%H%M")
hep_data_name = config.hep_data.split('/')[-1].split('.')[0]
output = "{0}/{1}".format( config.output_path,hep_data_name)
output += "/training-{0}/".format(date)

vb.INFO("RUN :  Saving output to {0}".format(output))
if not os.path.isdir(output):
    os.system( 'mkdir -p {0}'.format(output) )

 INFO :: RUN :  Saving output to ./example/training-23Oct2018-2325/


### Initialize & Setup Deep Learning class

In [5]:
dnn = Training() # class to do the training

In [6]:
dnn.variable_labels = plb.variable_labels()  # labels for variables
dnn.sample_labels   = plb.sample_labels()    # labels for samples

dnn.backend    = 'uproot'         # Set the backend for hepPlotter
dnn.hep_data   = config.hep_data
dnn.model_name = config.dnn_data
dnn.msg_svc    = vb
dnn.treename   = config.treename  # tree in Root file to access
dnn.useLWTNN   = True             # save model and architecture for LWTNN
dnn.dnn_name   = "dnn"
dnn.output_dim = config.output_dim
dnn.loss       = config.loss
dnn.init       = config.init
dnn.nNodes     = config.nNodes
dnn.metrics    = config.metrics
dnn.features   = config.features
dnn.epochs     = config.epochs
dnn.optimizer  = config.optimizer
dnn.input_dim  = len(config.features)
dnn.batch_size = config.batch_size
dnn.activations    = config.activation.split(',')
dnn.nHiddenLayers  = config.nHiddenLayers
dnn.earlystopping  = {'monitor':'loss','min_delta':0.0001,'patience':10,'mode':'auto'}
dnn.runDiagnostics = True            # make the performance plots
dnn.classes = {"bckg":0,"signal":1}  # dictionary of names with target values in ntuple
dnn.output_dir = output

NB: in the function call `dnn.train(ndims=X)`, `ndims` represents the number of dimensions to consider for plotting features for signal/background.  
- `ndims = -1`:    Plot 1D & 2D features for signal and background (can be slow)
- `ndims = 1`:     Plot 1D features only

### Run the training!

In [7]:
dnn.initialize()
dnn.load_data(['target'])  # load HEP data (add 'target' branch to dataframe)
dnn.preprocess_data()      # equal statistics for each class
dnn.train(ndims=1)         # build and train the model!

 INFO :: FOUNDATION : Load HEP data
 INFO :: FOUNDATION : -- pre-training :: features


/Users/demarley/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:1331: UserWarning: findfont: Font family [u'sans-serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))
/Users/demarley/Desktop/Home/hepex/hepPlotter/python/histogram1D.py:236: RuntimeWarning: divide by zero encountered in divide
  ratio_data.data.content = (num_data / np.sqrt(den_data)).copy()
/Users/demarley/Desktop/Home/hepex/hepPlotter/python/histogram1D.py:236: RuntimeWarning: invalid value encountered in divide
  ratio_data.data.content = (num_data / np.sqrt(den_data)).copy()
/Users/demarley/Desktop/Home/hepex/asimov/python/util.py:112: RuntimeWarning: invalid value encountered in divide
  tmp = np.divide( (sig-bkg)**2 , (sig+bkg), dtype=np.float32)
/Users/demarley/Desktop/Home/hepex/asimov/python/util.py:112: RuntimeWarning: divide by zero encountered in divide
  tmp = np.divide( (sig-bkg)**2 , (sig+bkg), dtype=np.float32)


 INFO :: FOUNDATION : -- pre-training :: correlations
 INFO :: FOUNDATION : -- pre-training :: separations
Train on 293538 samples, validate on 97846 samples
Epoch 1/10
293538/293538 [==============================] - 30s 101us/step - loss: 0.5009 - acc: 0.7547 - val_loss: 0.4798 - val_acc: 0.7717
Epoch 2/10
293538/293538 [==============================] - 26s 89us/step - loss: 0.4588 - acc: 0.7850 - val_loss: 0.4451 - val_acc: 0.7923
Epoch 3/10
293538/293538 [==============================] - 26s 89us/step - loss: 0.4381 - acc: 0.7971 - val_loss: 0.4308 - val_acc: 0.8026
Epoch 4/10
293538/293538 [==============================] - 26s 89us/step - loss: 0.4306 - acc: 0.8014 - val_loss: 0.4204 - val_acc: 0.8071
Epoch 5/10
293538/293538 [==============================] - 26s 89us/step - loss: 0.4267 - acc: 0.8025 - val_loss: 0.4363 - val_acc: 0.7960
Epoch 6/10
293538/293538 [==============================] - 26s 89us/step - loss: 0.4230 - acc: 0.8042 - val_loss: 0.4269 - val_acc: 0.8032
E

## Plots

Now the training is done!  
You can inspect the output plots to understand the performance of the neural network, then modify the configuration file and re-train, if necessary.
The model is saved in the output directory (by default to be used in the Lightweight NN framework).

In [8]:
from IPython.display import IFrame

In [9]:
IFrame("{0}acc_epochs.pdf".format(output), width=600, height=300) # doesn't handle '//' in filename

In [10]:
IFrame("{0}loss_epochs.pdf".format(output), width=600, height=300)

In [11]:
IFrame("{0}roc_curve.pdf".format(output), width=600, height=300)

In [12]:
IFrame("{0}hist_DNN_prediction.pdf".format(output), width=600, height=300)